In [ ]:
import torch 
import torchvision
import matplotlib.pyplot as plt 
import os 
import random

from PIL import Image
from torch import nn 
try:
  from torchinfo import summary
except ModuleNotFoundError as e:
  print(f"{e}, Downloading..")
  !pip install torchinfo
  from torchinfo import summary
from torchvision import transforms
from torch.utils.data import DataLoader

In [ ]:
!mkdir "./data" "./data/train" "./data/test" 

In [ ]:
BATCH_SIZE = 128
NUM_WORKERS = os.cpu_count()

train_transforms = transforms.Compose([
    transforms.Resize(size=(224, 224)),
    transforms.TrivialAugmentWide(num_magnitude_bins=31),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize(size=(224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_data = torchvision.datasets.CIFAR100(root="./data/train",
                                     train=True,
                                     transform=train_transforms,
                                     download=True)

test_data = torchvision.datasets.CIFAR100(root="./data/test",
                                          train=False,
                                          transform=test_transforms,
                                          download=True)

train_dataloader = DataLoader(dataset=train_data,
                              batch_size=BATCH_SIZE,
                              num_workers=NUM_WORKERS,
                              shuffle=True,
                              pin_memory=True,
                              generator=torch.Generator(device="cpu"))

test_dataloader = DataLoader(dataset=test_data,
                             batch_size=BATCH_SIZE,
                             num_workers=NUM_WORKERS,
                             shuffle=False,
                             pin_memory=True,
                             generator=torch.Generator(device="cpu"))

class_names = train_data.classes

len(train_dataloader), len(test_dataloader), class_names

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
imgs, labels = next(iter(train_dataloader))
imgs.shape, labels.shape

In [ ]:
import random
random_imgs = random.sample(range(0, 128), k=5)

fig, ax = plt.subplots(nrows=1, ncols=5, sharex=True, sharey=True, figsize=(14,10))
for idx, img in enumerate(random_imgs):
  ax[idx].imshow(imgs[img].permute(1, 2, 0))
  ax[idx].set_title(f"{class_names[labels[img]]}")
  ax[idx].axis("off");

In [ ]:
model_ft = torchvision.models.resnet50(weights="DEFAULT")
for param in model_ft.parameters():
  param.requires_grad=False
for param in model_ft.layer4.parameters():
  param.requires_grad = True
in_features = model_ft.fc.in_features
model_ft.fc = nn.Linear(in_features, 100)
model_ft = model_ft.to(device)
loss_fn = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = torch.optim.Adam(model_ft.parameters(), lr=0.001)



In [ ]:
summary(model=model_ft,
        input_size=(1, 3, 224, 224), # (batch_size, color_channels, height, width)
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

In [ ]:
def train_model(model, train_dataloader, test_dataloader, loss_fn, optimizer, num_epochs, device):
  results = {"train_loss":[],
             "train_acc":[],
             "test_acc":[],
             "test_loss":[]}

  for epoch in range(num_epochs):
    train_loss, train_acc = 0, 0
    model.train()

    for batch, (X, y) in enumerate(train_dataloader):
      X, y = X.to(device), y.to(device)
      y_preds = model(X)
      trn_loss = loss_fn(y_preds, y)
      train_loss += trn_loss.item()
      optimizer.zero_grad()
      trn_loss.backward()
      optimizer.step()

      y_class_preds = torch.argmax(torch.softmax(y_preds, dim=1), dim=1)
      train_acc += (y_class_preds==y).sum().item() / len(y_preds)

    train_loss = train_loss / len(train_dataloader)
    train_acc = train_acc / len(train_dataloader)

    test_loss, test_acc = 0, 0
    model.eval()
    with torch.inference_mode():
      for batch, (X, y) in enumerate(test_dataloader):
        X, y = X.to(device), y.to(device)
        test_preds = model(X)
        tst_loss = loss_fn(test_preds, y)
        test_loss += tst_loss.item()

        test_pred_labels = torch.argmax(test_preds, dim=1)
        test_acc += (test_pred_labels == y).sum().item() / len(test_preds)

      test_loss = test_loss / len(test_dataloader)
      test_acc = test_acc / len(test_dataloader)

    print(
      f"Epoch: {epoch+1} | "
      f"train_loss: {train_loss:.4f} | "
      f"train_acc: {train_acc:.4f} | "
      f"test_loss: {test_loss:.4f} | "
      f"test_acc: {test_acc:.4f}"
      )
    results["train_loss"].append(train_loss)
    results["train_acc"].append(train_acc)
    results["test_loss"].append(test_loss)
    results["test_acc"].append(test_acc)
  return results

In [ ]:
results = train_model(model=model_ft,
                      train_dataloader=train_dataloader,
                      test_dataloader=test_dataloader,
                      loss_fn=loss_fn,
                      optimizer=optimizer,
                      num_epochs=10,
                      device=device)

In [ ]:
model_ft_with_all_layers_frozen = torchvision.models.resnet50(weights="DEFAULT")
for param in model_ft_with_all_layers_frozen.parameters():
  param.requires_grad=False
in_features = model_ft_with_all_layers_frozen.fc.in_features
model_ft_with_all_layers_frozen.fc = nn.Linear(in_features, 100)
model_ft_with_all_layers_frozen = model_ft_with_all_layers_frozen.to(device)
loss_fn = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = torch.optim.Adam(model_ft_with_all_layers_frozen.parameters(), lr=0.001)

In [ ]:
summary(model=model_ft_with_all_layers_frozen,
        input_size=(1, 3, 224, 224), # (batch_size, color_channels, height, width)
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

In [ ]:
results_with_all_layers_frozen = train_model(model=model_ft_with_all_layers_frozen,
                      train_dataloader=train_dataloader,
                      test_dataloader=test_dataloader,
                      loss_fn=loss_fn,
                      optimizer=optimizer,
                      num_epochs=10,
                      device=device)

In [ ]:
import pandas as pd 
df_layer4_unfrozen = pd.DataFrame(results)
df_layer4_unfrozen.plot(title="CIFAR100 Fine Tuned Results")

In [ ]:
df_all_frozen = pd.DataFrame(results_with_all_layers_frozen)
df_all_frozen.plot(title="CIFAR100 Feature Extractor Results")